In [93]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [94]:
df1 = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau.csv")
df1.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629


In [95]:
df2 = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau_balance.csv")
df2.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [96]:
merge_df = pd.merge(df1, df2, on='SK_ID_BUREAU')
merge_df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR,MONTHS_BALANCE,STATUS
0,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,0,C
1,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-1,C
2,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-2,C
3,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-3,C
4,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-4,C


In [97]:
app_train = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_application_train.csv")
app_train = app_train[['SK_ID_CURR','TARGET']]
merged_df = pd.merge(merge_df,app_train, on='SK_ID_CURR',how = 'inner')

In [98]:
merged_df

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR,MONTHS_BALANCE,STATUS,TARGET
0,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,0,C,0
1,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-1,C,0
2,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-2,C,0
3,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-3,C,0
4,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,137415,-4,C,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11769864,5053356,Closed,currency 1,-2724,0,-2511.0,-2511.0,0.0,0,101250.0,0.0,0.0,0.0,Consumer credit,-2510,NaN,139955,-85,0,0
11769865,5053356,Closed,currency 1,-2724,0,-2511.0,-2511.0,0.0,0,101250.0,0.0,0.0,0.0,Consumer credit,-2510,NaN,139955,-86,0,0
11769866,5053356,Closed,currency 1,-2724,0,-2511.0,-2511.0,0.0,0,101250.0,0.0,0.0,0.0,Consumer credit,-2510,NaN,139955,-87,0,0
11769867,5053356,Closed,currency 1,-2724,0,-2511.0,-2511.0,0.0,0,101250.0,0.0,0.0,0.0,Consumer credit,-2510,NaN,139955,-88,0,0


# Features Engineering and Selection

### Numerical Data

#### drop the DAYS_ENDDATE_FACT column
#### convert numbers to positive numbers

In [99]:
merged_df = merged_df.drop('DAYS_ENDDATE_FACT', axis=1)
merged_df['DAYS_CREDIT'] = merged_df['DAYS_CREDIT'].fillna(0).astype(int).abs()
merged_df['DAYS_CREDIT_ENDDATE'] = merged_df['DAYS_CREDIT_ENDDATE'].fillna(0).astype(int).abs()
merged_df['DAYS_CREDIT_UPDATE'] = merged_df['DAYS_CREDIT_UPDATE'].fillna(0).astype(int).abs()

### Categorical data

#### drop the CREDIT_CURRENCY column

In [100]:
merged_df = merged_df.drop(['CREDIT_CURRENCY'], axis=1)

In [101]:
merged_df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR,MONTHS_BALANCE,STATUS,TARGET
0,5715448,Active,820,0,31069,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,183,0.0,137415,0,C,0
1,5715448,Active,820,0,31069,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,183,0.0,137415,-1,C,0
2,5715448,Active,820,0,31069,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,183,0.0,137415,-2,C,0
3,5715448,Active,820,0,31069,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,183,0.0,137415,-3,C,0
4,5715448,Active,820,0,31069,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,183,0.0,137415,-4,C,0


In [102]:
num_data = merged_df._get_numeric_data()
num_col = num_data.columns
cat_data = merged_df[[col for col in merged_df.columns if col not in num_data.columns]]

In [103]:
#group by sk_id_curr
#sum, min, max, count tao bang moi

In [104]:
merged_df.columns

Index(['SK_ID_BUREAU', 'CREDIT_ACTIVE', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT_ENDDATE', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY', 'SK_ID_CURR', 'MONTHS_BALANCE', 'STATUS', 'TARGET'],
      dtype='object')

#### SK_ID_CURR column is duplicated. So we group by all data and use sum, min, max count to create new table

In [105]:
num_data = num_data.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_MAX_OVERDUE': ['mean','min','max','sum'],
    'AMT_CREDIT_SUM': ['mean', 'sum', 'min', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum', 'min', 'max'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum', 'min', 'max'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean', 'sum', 'min', 'max'],
    'AMT_ANNUITY': ['mean', 'sum', 'min', 'max']
})
num_data.columns = ['_'.join(col).strip().upper() for col in num_data.columns.values]
num_data

,AMT_CREDIT_MAX_OVERDUE_MEAN,AMT_CREDIT_MAX_OVERDUE_MIN,AMT_CREDIT_MAX_OVERDUE_MAX,AMT_CREDIT_MAX_OVERDUE_SUM,AMT_CREDIT_SUM_MEAN,AMT_CREDIT_SUM_SUM,AMT_CREDIT_SUM_MIN,AMT_CREDIT_SUM_MAX,AMT_CREDIT_SUM_DEBT_MEAN,AMT_CREDIT_SUM_DEBT_SUM,...,AMT_CREDIT_SUM_LIMIT_MIN,AMT_CREDIT_SUM_LIMIT_MAX,AMT_CREDIT_SUM_OVERDUE_MEAN,AMT_CREDIT_SUM_OVERDUE_SUM,AMT_CREDIT_SUM_OVERDUE_MIN,AMT_CREDIT_SUM_OVERDUE_MAX,AMT_ANNUITY_MEAN,AMT_ANNUITY_SUM,AMT_ANNUITY_MIN,AMT_ANNUITY_MAX
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
1,11666.385000,11666.385,11666.385,629984.790,2.835000e+04,1.530900e+06,28350.0,28350.0,0.000000,0.000000e+00,...,0.0,0.000,0.0,0.0,0.0,0.0,NaN,0.00,NaN,NaN
8,NaN,NaN,NaN,0.000,2.251714e+06,1.981509e+08,157500.0,2351439.0,0.000000,0.000000e+00,...,0.0,0.000,0.0,0.0,0.0,0.0,8439.809318,742703.22,1967.805,8748.00
11,NaN,NaN,NaN,0.000,1.766894e+05,7.138253e+07,13500.0,697500.0,16757.056886,5.596857e+06,...,0.0,0.000,0.0,0.0,0.0,0.0,5929.600000,747129.60,0.000,26683.20
20,0.000000,0.000,0.000,0.000,1.890489e+05,8.696250e+06,0.0,342000.0,91200.521739,4.195224e+06,...,0.0,0.000,0.0,0.0,0.0,0.0,11659.832609,536352.30,10084.500,12500.01
21,129.544535,0.000,186.705,16711.245,1.718247e+05,8.007029e+07,0.0,540000.0,59165.710079,2.632874e+07,...,0.0,227915.685,0.0,0.0,0.0,0.0,0.000000,0.00,0.000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307494,0.000000,0.000,0.000,0.000,6.009827e+04,1.298123e+07,0.0,382500.0,4802.224500,7.683559e+05,...,0.0,41977.755,0.0,0.0,0.0,0.0,0.000000,0.00,0.000,0.00
307496,NaN,NaN,NaN,0.000,6.405163e+04,2.946375e+06,51705.0,78750.0,23027.184783,1.059250e+06,...,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000,0.00
307501,NaN,NaN,NaN,0.000,3.474111e+05,5.419613e+07,73045.8,2250000.0,325308.150000,3.903698e+07,...,0.0,0.000,0.0,0.0,0.0,0.0,24750.000000,222750.00,24750.000,24750.00


In [106]:
num_col = list(num_col)
num_col.remove('SK_ID_CURR')

cat_df = df[[col for col in df.columns if col not in num_col]]
cat_df = cat_df.groupby(['SK_ID_CURR']).agg(sum)

In [107]:
final_bureau = pd.merge(num_data,cat_df,left_index=True, right_index=True)
final_bureau.head()

,AMT_CREDIT_MAX_OVERDUE_MEAN,AMT_CREDIT_MAX_OVERDUE_MIN,AMT_CREDIT_MAX_OVERDUE_MAX,AMT_CREDIT_MAX_OVERDUE_SUM,AMT_CREDIT_SUM_MEAN,AMT_CREDIT_SUM_SUM,AMT_CREDIT_SUM_MIN,AMT_CREDIT_SUM_MAX,AMT_CREDIT_SUM_DEBT_MEAN,AMT_CREDIT_SUM_DEBT_SUM,...,AMT_CREDIT_SUM_OVERDUE_MEAN,AMT_CREDIT_SUM_OVERDUE_SUM,AMT_CREDIT_SUM_OVERDUE_MIN,AMT_CREDIT_SUM_OVERDUE_MAX,AMT_ANNUITY_MEAN,AMT_ANNUITY_SUM,AMT_ANNUITY_MIN,AMT_ANNUITY_MAX,CREDIT_ACTIVE,CREDIT_TYPE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
1,11666.385000,11666.385,11666.385,629984.790,2.835000e+04,1.530900e+06,28350.0,28350.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,NaN,0.00,NaN,NaN,Active,Consumer credit
8,NaN,NaN,NaN,0.000,2.251714e+06,1.981509e+08,157500.0,2351439.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,8439.809318,742703.22,1967.805,8748.00,ClosedSold,Consumer creditCar loan
11,NaN,NaN,NaN,0.000,1.766894e+05,7.138253e+07,13500.0,697500.0,16757.056886,5.596857e+06,...,0.0,0.0,0.0,0.0,5929.600000,747129.60,0.000,26683.20,ClosedClosedClosedClosedActiveActiveClosedClos...,Consumer creditCredit cardConsumer creditConsu...
20,0.000000,0.000,0.000,0.000,1.890489e+05,8.696250e+06,0.0,342000.0,91200.521739,4.195224e+06,...,0.0,0.0,0.0,0.0,11659.832609,536352.30,10084.500,12500.01,ClosedActiveActiveClosed,Consumer creditCredit cardConsumer creditConsu...
21,129.544535,0.000,186.705,16711.245,1.718247e+05,8.007029e+07,0.0,540000.0,59165.710079,2.632874e+07,...,0.0,0.0,0.0,0.0,0.000000,0.00,0.000,0.00,ActiveActiveActiveActiveClosedClosedActiveClos...,Credit cardCredit cardCredit cardCredit cardCo...


## Null and Outliers

In [108]:
final_bureau.isnull().sum() / final_bureau.shape[0]

AMT_CREDIT_MAX_OVERDUE_MEAN    0.329745
AMT_CREDIT_MAX_OVERDUE_MIN     0.329745
AMT_CREDIT_MAX_OVERDUE_MAX     0.329745
AMT_CREDIT_MAX_OVERDUE_SUM     0.000000
AMT_CREDIT_SUM_MEAN            0.000000
AMT_CREDIT_SUM_SUM             0.000000
AMT_CREDIT_SUM_MIN             0.000000
AMT_CREDIT_SUM_MAX             0.000000
AMT_CREDIT_SUM_DEBT_MEAN       0.025728
AMT_CREDIT_SUM_DEBT_SUM        0.000000
AMT_CREDIT_SUM_DEBT_MIN        0.025728
AMT_CREDIT_SUM_DEBT_MAX        0.025728
AMT_CREDIT_SUM_LIMIT_MEAN      0.088056
AMT_CREDIT_SUM_LIMIT_SUM       0.000000
AMT_CREDIT_SUM_LIMIT_MIN       0.088056
AMT_CREDIT_SUM_LIMIT_MAX       0.088056
AMT_CREDIT_SUM_OVERDUE_MEAN    0.000000
AMT_CREDIT_SUM_OVERDUE_SUM     0.000000
AMT_CREDIT_SUM_OVERDUE_MIN     0.000000
AMT_CREDIT_SUM_OVERDUE_MAX     0.000000
AMT_ANNUITY_MEAN               0.132377
AMT_ANNUITY_SUM                0.000000
AMT_ANNUITY_MIN                0.132377
AMT_ANNUITY_MAX                0.132377
CREDIT_ACTIVE                  0.000000


%null is low <5% -> Can remove all row null

In [109]:
final_bureau = final_bureau.dropna()

## Transformation